# 科学技術計算 6 レポート


In [6]:
import numpy as np
import scipy 
from scipy.optimize import OptimizeResult

from numpy.linalg import inv, norm, solve, det, matrix_rank, cond
from numpy import diag
rng = np.random.default_rng()

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"
from matplotlib.colors import BoundaryNorm

from typing import List, Optional, Union, Tuple, Dict

import jax.numpy as jnp


ModuleNotFoundError: No module named 'jax'

## 課題06-1

In [ ]:
#目的関数を作成
class AnotherNonConvexFunctionJax(FunctionJax):
    def forward(self, x: jnp.ndarray) -> float:
        x0, x1 = x[0], x[1]
        return (
            0.05 * (x0**4 + x1**4)
            - (x0**2 + x1**2)
            + 0.5 * (x0 + x1)
        )




NameError: name 'FunctionJax' is not defined

In [ ]:
class CacheXk1D():
    """1次元のスカラー値をキャッシュするクラス"""

    def is_scalar(
            self,
            x: float | int | np.ndarray
        ) -> bool:
        """xがfloat, int, または単一要素のndarrayかを確認する"""
        return isinstance(x, float) \
            or isinstance(x, int) \
            or (isinstance(x, np.ndarray) \
                and x.ndim == 1 \
                and len(x) == 1)

    def __init__(
            self,
            f: callable,
            g: callable,
            x0: float | int | np.ndarray,
            name: str
        ) -> None:
        """コンストラクタ

        Args:
            f (callable, ``f(x: float)``): 目的関数 f
            g (callable, ``g(x: float)``): 導関数 f'
            x0 (float | np.ndarray): 初期値
            name (str): キャッシュ名
        """
        assert self.is_scalar(x0), "x must be float or int"
        self.__x_series = [x0]
        self.__f_series = [f(x0)]
        self.__g_series = [abs(g(x0))]
        self.f = f
        self.g = g
        self.__name = str(name)

    def __call__(self, xk: float) -> None:
        """xkをキャッシュに追加するコールバック

        Args:
            xk (float): 現在の解 x_k
        """
        assert self.is_scalar(xk), "x must be float or int"
        self.__x_series.append(xk)
        self.__f_series.append(self.f(xk))
        self.__g_series.append(abs(self.g(xk)))

    def get_x(self) -> np.ndarray:
        """x0, x1, ..., xkのリストを取得する"""
        return np.array(self.__x_series)

    def get_f(self) -> np.ndarray:
        """f(x0), f(x1), ..., f(xk)のリストを取得する"""
        return np.array(self.__f_series)

    def get_g(self) -> np.ndarray:
        """|f'(x0)|, |f'(x1)|, ..., |f'(xk)|のリストを取得する"""
        return np.array(self.__g_series)

    def get_name(self) -> str:
        """キャッシュ名を取得する"""
        return str(self.__name)

    def set_name(self, name:str) -> None:
        """キャッシュ名を設定する"""
        self.__name = str(name)

In [ ]:
def grad_decent_1d(
        f: callable,
        g: callable,
        x0: float,
        alpha: float = 0.1,
        maxiter: int = 100,
        callback: callable = None
) -> float:
    """1次元関数の勾配法による最小化（コールバック版）

    Args:
        f (callable, ``f(x: float)``): 目的関数 f
        g (callable, ``g(x: float)``): 導関数 f'
        x0 (float): 初期値
        alpha (float, optional): ステップサイズ．デフォルトは0.1
        maxiter (int, optional): 最大反復回数．デフォルトは100
        callback (callable, ``callback(x: float)``, optional): コールバック関数．デフォルトはNone

    Returns:
        float: 最適解 \hat{x}
    """
    assert isinstance(x0, float)

    x = x0
    for k in range(maxiter):
        x = x - alpha * g(x)  # 最急降下法の更新式

        # コールバック関数が指定されている場合、現在のxをコールバックに渡す
        if callback:
            callback(x)

        # 終了条件：勾配の絶対値が小さい場合に終了
        if np.isclose(np.abs(g(x)), 0):
            return x

    return x


## 課題6-2

In [ ]:
def quasi_newton(
        f: callable,
        g: callable,
        x0: np.ndarray,
        update: str,  # "B" or "H"
        method: str,  # "BFGS", "DFP", "SR1"
        maxiter: int,
        callback: callable,
) -> np.ndarray: